# Simulation for MSCI 333 Project
By: Justin Du, Michael Sheng

In [ ]:
!pip install simpy

import simpy
import random
import numpy as np

class SFGHSimulation:
    def __init__(self, env):
        self.env = env
        self.ambulance_queue = simpy.Resource(env, capacity=1)
        self.walk_in_queue = simpy.Resource(env, capacity=1)
        self.waiting_room = simpy.Resource(env, capacity=100)       #Combined waiting room for Non-Urgent and Referred
        self.resuscitation_zone = simpy.Resource(env, capacity=1)   # Resuscitation zone with a capacity of 1 patient
        self.zone_1R = simpy.Resource(env, capacity=8)              # Zone 1R which is exclusive for resuscitation patients has a capacity of 8 patients
        self.zone_1 = simpy.Resource(env, capacity=18)              # Zone 1 with a capacity of 18 patients
        self.zone_2 = simpy.Resource(env, capacity=8)               # Zone 2 with a capacity of 8 patients
        self.zone_3 = simpy.Resource(env, capacity=12)              # Zone 3 with a capacity of 12 patients
        self.zone_4 = simpy.Resource(env, capacity=12)              # Zone 4 with a capacity of 12 patients
        self.patients_in_waiting_room = 0
        self.critical_care_diversion = False

    def patient_arrival(self):
        return random.uniform[0,1] <= np.random.poisson(1/12)

    #arriving by ambulance
    def arrival_type_ambulance(self, patient):
        trauma_probability = 0.5
        is_trauma = random.uniform(0, 1) < trauma_probability

        with self.ambulance_queue.request() as request:
            yield request
            print(f"Patient {patient} arrived by ambulance at time {self.env.now}.")

            #trauma patients assumed to be traige level resuscitation
            if is_trauma:
                print(f"Patient {patient} is a traum patient, cannot be diverted.")
                with self.zone_1R.request() as zone_request: #request methods requests a capacity spot, keeps track of zone capacity behind the scenes
                    try:
                        yield zone_request
                        print(f"Patient {patient} moved to Zone 1R at time {self.env.now}.")
                        time_spent_zone_1R = random.uniform(35, 180)
                        yield self.env.timeout(time_spent_zone_1R)
                        print(f"Patient {patient} spent {time_spent_zone_1R} minutes in Zone 1R.")


                        # when admitted they exit our system so they are also discharged
                        discharge_time = self.env.now
                        print(f"Patient {patient} discharged at time {discharge_time}.")

                    except simpy.Interrupt:
                        print(f"Critical Care Diversion: Patient {patient} since all resuscitation beds are full.")
                        self.critical_care_diversion = True
                        return

            #patients are not trauma patients, can be triage level 2-5
            else:
                print(f"Patient {patient} categorized as Non-Trauma.")

                trauma_catgories = ["Emergency", "Resuscitation", "Urgent", "Non-Urgent"]

                non_trauma_label = random.choices(trauma_catgories, weights = [20, 35, 30, 15])


                if non_trauma_label == "Emergency":
                    print(f"Patient {patient} labelled as Emergency.")
                    triage_duration_emergency = random.uniform(1, 2)
                    yield self.env.timeout(triage_duration_emergency)

                    with self.zone_1.request() as zone_request:
                        try:
                            yield zone_request
                            print(f"Patient {patient} moved to Zone 1 at time {self.env.now}.")
                            zone_1_time = random.uniform(35, 180)
                            yield self.env.timeout(zone_1_time)
                            print(f"Patient {patient} spent {zone_1_time} minutes in Zone 1.")

                            discharge_time = self.env.now
                            print(f"Patient {patient} discharged at time {discharge_time}.")

                        except simpy.Interrupt:
                            self.redirect_between_zones(patient, self.zone_1, self.zone_2)

                elif non_trauma_label == "Resuscitation":
                    print(f"Patient {patient} labelled as Resuscitation.")
                    triage_duration_resuscitation = random.uniform(1, 2)
                    yield self.env.timeout(triage_duration_resuscitation)

                    with self.resuscitation_zone.request() as resuscitation_request:
                        try:
                            yield resuscitation_request
                            print(f"Patient {patient} moved to Resuscitation at time {self.env.now}.")

                            resuscitation_zone_time = random.uniform(35, 180)
                            yield self.env.timeout(resuscitation_zone_time)
                            print(f"Patient {patient} spent {resuscitation_zone_time} minutes in Resuscitation.")

                            # Discharge
                            discharge_time = self.env.now
                            print(f"Patient {patient} discharged at time {discharge_time}.")

                        except simpy.Interrupt:
                            self.redirect_between_zones(patient, self.resuscitation_zone, self.zone_1)

                elif non_trauma_label == "Urgent":
                    print(f"Patient {patient} labelled as Urgent.")
                    triage_duration_urgent = random.uniform(1, 2)
                    yield self.env.timeout(triage_duration_urgent)

                    #50% chance to send patient to Zone 1
                    if random.uniform(0, 1) < 0.5:
                        with self.zone_1.request() as zone_request:
                            try:
                                yield zone_request
                                print(f"Patient {patient} moved to Zone 1 at time {self.env.now}.")

                                zone_1_workup_time = random.uniform(35, 180)
                                yield self.env.timeout(zone_1_workup_time)
                                print(f"Patient {patient} spent {zone_1_workup_time} minutes in Zone 1.")

                                #patient is admitted after work up in zone 1 so they leave our system
                                discharge_time = self.env.now
                                print(f"Patient {patient} discharged at time {discharge_time}.")

                            # if zone 1 is full then redirect
                            except simpy.Interrupt:
                                self.redirect_between_zones(patient, self.zone_1, self.zone_2)

                    else:
                        # 50% chance to send patient to Zone 2
                        with self.zone_2.request() as zone_request:
                            try:
                                yield zone_request
                                print(f"Patient {patient} moved to Zone 2 at time {self.env.now}.")

                                zone_2_workup_time = random.uniform(35, 180)
                                yield self.env.timeout(zone_2_workup_time)
                                print(f"Patient {patient} spent {zone_2_workup_time} minutes in Zone 2.")

                                discharge_time = self.env.now
                                print(f"Patient {patient} discharged at time {discharge_time}.")

                            except simpy.Interrupt:
                                self.redirect_between_zones(patient, self.zone_2, self.zone_1)

                else:  # Non-Urgent
                    print(f"Patient {patient} labelled as Non-Urgent.")
                    triage_duration_nonurgent = random.uniform(10, 15)
                    yield self.env.timeout(triage_duration_nonurgent)

                    with self.waiting_room.request() as wait_request:
                        yield wait_request
                        self.patients_in_waiting_room += 1
                        print(f"Patient {patient} moved to Waiting Room at time {self.env.now}. "
                              f"Number of people in the waiting room: {self.patients_in_waiting_room}")

                    # Remaining logic for Zone 3 and Zone 4 is unchanged

            assigned_time = self.env.now
            print(f"Patient {patient} assigned category at time {assigned_time}")

    #redirect non resuscitation patients between zones 1 and 2
    def redirect_between_zones(self, patient, zone_from, zone_to):
        with zone_to.request() as zone_request:
            try:
                yield zone_request
                print(f"Patient {patient} moved to {zone_to.name} from {zone_from.name} at time {self.env.now}.")
                zone_1_2_workup_time = random.uniform(35, 180)
                yield self.env.timeout(zone_time)
                print(f"Patient {patient} spent {zone_1_2_workup_time} minutes in {zone_to.name}.")

                discharge_time = self.env.now
                print(f"Patient {patient} discharged at time {discharge_time}.")

            except simpy.Interrupt:
                print(f"Patient {patient} redirected due to {zone_to.name} being full.")
                return

    def arrival_type_walk_in(self, patient):
        arrival_time = self.env.now
        triage_categories = ["Urgent", "Non-Urgent", "Referred"]

        triage_level = random.choices(triage_categories, weights = [33, 43, 23])


        with self.walk_in_queue.request() as request:
            yield request
            time_taken = 0  # Placeholder for time taken based on the category

            if triage_level == "Urgent":
                triage_duration = random.uniform(1, 2)  # Urgent process time between 1 and 2 minutes
                yield self.env.timeout(triage_duration)

                #50% chance in sending patient to zone 3
                if random.uniform(0, 1) < 0.5:
                    with self.zone_3.request() as zone_request:
                        try:
                            yield zone_request
                            print(f"Patient {patient} moved to Zone 3 at time {self.env.now}.")
                            zone_3_workup_time = random.uniform(35, 180)
                            yield self.env.timeout(zone_3_workup_time)
                            print(f"Patient {patient} spent {zone_3_workup_time} minutes in Zone 3.")

                            discharge_time = self.env.now
                            print(f"Patient {patient} discharged at time {discharge_time}.")

                        except simpy.Interrupt:
                            print(f"Patient {patient} redirected due to Zone 3 being full.")
                            return

                else:
                    #50% chance in sending patient to zone 3
                    with self.zone_4.request() as zone_request:
                        try:
                            yield zone_request
                            print(f"Patient {patient} moved to Zone 4 at time {self.env.now}.")
                            zone_4_workup_time = random.uniform(35, 180)
                            yield self.env.timeout(zone_4_workup_time)
                            print(f"Patient {patient} spent {zone_4_workup_time} minutes in Zone 4.")

                            discharge_time = self.env.now
                            print(f"Patient {patient} discharged at time {discharge_time}.")

                        except simpy.Interrupt:
                            print(f"Patient {patient} redirected due to Zone 4 being full.")
                            return

            else:
                #patient has treiage level non-urgent or referred
                triage_duration = random.uniform(10, 15)  #process time between 10 and 15 minutes

                yield self.env.timeout(triage_duration)
                with self.waiting_room.request() as wait_request:
                    yield wait_request
                    self.patients_in_waiting_room += 1
                    print(f"Patient {patient} moved to Waiting Room at time {self.env.now}. "
                          f"Number of people in the waiting room: {self.patients_in_waiting_room}")

                    #50% chance in sending patient to Zone 3
                    if random.uniform(0, 1) < 0.5:
                        yield self.env.timeout(0)  #patient is moved our fgrom waiting room
                        with self.zone_3.request() as zone_request:
                            try:
                                yield zone_request
                                self.patients_in_waiting_room -= 1
                                print(f"Patient {patient} moved to Zone 3 at time {self.env.now}. "
                                      f"Number of people in the waiting room: {self.patients_in_waiting_room}")

                                zone_3_workup_time = random.uniform(20, 90)
                                yield self.env.timeout(zone_3_workup_time)
                                print(f"Patient {patient} spent {zone_3_workup_time} minutes in Zone 3.")

                                discharge_time = self.env.now
                                print(f"Patient {patient} discharged at time {discharge_time}.")

                            except simpy.Interrupt:
                                print(f"Patient {patient} redirected due to Zone 3 being full.")
                                return
                    else:
                        # 50% chance to send patient to Zone 4
                        yield self.env.timeout(0)  ##patient is moved our from waiting room
                        with self.zone_4.request() as zone_request:
                            try:
                                yield zone_request
                                self.patients_in_waiting_room -= 1
                                print(f"Patient {patient} moved to Zone 4 at time {self.env.now}. "
                                      f"Number of people in the waiting room: {self.patients_in_waiting_room}")

                                zone_4_workup_time = random.uniform(20, 90)
                                yield self.env.timeout(zone_4_workup_time)
                                print(f"Patient {patient} spent {zone_4_workup_time} minutes in Zone 4.")

                                discharge_time = self.env.now
                                print(f"Patient {patient} discharged at time {discharge_time}.")

                            except simpy.Interrupt:
                                print(f"Patient {patient} redirected due to Zone 4 being full.")
                                return

            assigned_time = arrival_time + time_taken
            print(f"Patient {patient} arrived as walk-in ({triage_level}) at time {arrival_time}")
            print(f"Patient {patient} assigned category at time {assigned_time}")

def patient_generator(env, ed):
    patient_count = 1
    while env.now < 24 * 60:  # Continue generating patients until 24 hours
        if ed.patient_arrival:
          if not ed.critical_care_diversion:
              #assumed 75% chance of walk in
              if random.uniform(0, 1) < 0.75:
                  env.process(ed.arrival_type_walk_in(patient_count))
              else:
                  #assumed 25% chance of ambulance
                  env.process(ed.arrival_type_ambulance(patient_count))
              patient_count += 1
          yield env.timeout(1)


def run_simulation():
    env = simpy.Environment()
    emergency_department = SFGHSimulation(env)
    env.process(patient_generator(env, emergency_department))

    total_simulation_time = 24*60*7
    env.run(until=total_simulation_time)  #Run the simulation for 1 full week


if __name__ == "__main__":
    run_simulation()


Patient 2 arrived by ambulance at time 1.
Patient 2 is a traum patient, cannot be diverted.
Patient 2 moved to Zone 1R at time 1.
Patient 1 moved to Waiting Room at time 13.46085378446452. Number of people in the waiting room: 1
Patient 1 moved to Zone 4 at time 13.46085378446452. Number of people in the waiting room: 0
Patient 1 spent 20.30149003416863 minutes in Zone 4.
Patient 1 discharged at time 33.76234381863315.
Patient 1 arrived as walk-in (['Referred']) at time 0
Patient 1 assigned category at time 0
Patient 4 moved to Waiting Room at time 48.10764118274322. Number of people in the waiting room: 1
Patient 4 moved to Zone 4 at time 48.10764118274322. Number of people in the waiting room: 0
Patient 2 spent 92.76573017300828 minutes in Zone 1R.
Patient 2 discharged at time 93.76573017300828.
Patient 2 assigned category at time 93.76573017300828
Patient 3 arrived by ambulance at time 93.76573017300828.
Patient 3 categorized as Non-Trauma.
Patient 3 labelled as Non-Urgent.
Patient 